<a href="https://colab.research.google.com/github/stevejj4/SHOFCO-SUN/blob/master/Test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

# Explicitly authenticate user
drive.mount('/content/drive', force_remount=True) # Added force_remount=True to ensure fresh authentication

Mounted at /content/drive


In [ ]:
import pandas as pd

# Replace these paths with the paths to your uploaded files
members_path = '/content/drive/MyDrive/Report processing test/Members.csv'
payments_path = '/content/drive/MyDrive/Report processing test/PaymentsJan.xlsx'

members = pd.read_csv(members_path)
payments = pd.read_excel(payments_path) # Use pd.read_excel for Excel files

<ipython-input-34-5d40eb06b3f1>:7: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  members = pd.read_csv(members_path)


In [ ]:
payments.head()

,PAY BILL NO,PAYEMENT DATE,AMOUNT PAID,PHONE NO USED TO PAY,Account no
0,TA17JNVTLT,2025-01-01 00:02:00,100,254721***525 - THOMAS ***,6369908.0
1,TA18JO8WSM,2025-01-01 00:06:00,100,072903***542 - bendict ***,8617384.0
2,TA11JPXPS1,2025-01-01 00:24:00,200,254718***825 - MARTHA ***,30368127.0
3,TA16JR13D8,2025-01-01 00:38:00,600,254720***967 - ESTHER ***,13225946.0
4,TA15JTNUXL,2025-01-01 01:18:00,100,072294***816 - Benard ***,42728706.0


In [ ]:
# Convert 'ID Number', 'Account no', 'NationalID', and 'Account no' to numeric if needed
members['ID Number'] = pd.to_numeric(members['ID Number'], errors='coerce')
# Replace 'ID Number' with the actual column name for National ID in your members DataFrame
members['NationalID'] = pd.to_numeric(members['ID Number'], errors='coerce')
payments['Account no'] = pd.to_numeric(payments['Account no'], errors='coerce')
# Assuming 'Account no' is the column representing AccountNumber in payments DataFrame
payments['Account no'] = pd.to_numeric(payments['Account no'], errors='coerce')

# Merge the datasets on NationalID/Account no
merged_data = pd.merge(members, payments, left_on='NationalID', right_on='Account no') # Changed column names for merge

merged_data.columns


Index(['ID', 'Registered By', 'Registration Date', 'Member Name',
       'Primary Phone Number', 'ID Type', 'ID Number', 'Status',
       'Policy Matured', 'Ward', 'Group Name', 'Join Type', 'NationalID',
       'PAY BILL NO', 'PAYEMENT DATE', 'AMOUNT PAID', 'PHONE NO USED TO PAY',
       'Account no'],
      dtype='object')

In [ ]:
# total payments per ward
total_payments_per_ward = merged_data.groupby('Ward')['AMOUNT PAID'].sum()
total_payments_per_ward
# download to excel
total_payments_per_ward.to_excel('total_payments_per_ward.xlsx')

In [45]:
# Group by 'Group Name' and 'Ward', then sum 'AMOUNT PAID' and get member counts
grouped_data = merged_data.groupby(['Group Name', 'Ward']).agg(
    Total_Amount_Paid=('AMOUNT PAID', 'sum'),
    Total_Members_Paid=('NationalID', 'nunique')  # Count unique NationalIDs
).reset_index()

# Merge with member_counts to get 'Total Member Count'
final_group_totals = pd.merge(grouped_data, member_counts, on='Group Name', how='left')

# Reorder columns
final_group_totals = final_group_totals[['Group Name', 'Ward', 'Total Member Count', 'Total_Members_Paid', 'Total_Amount_Paid']]

# Display the updated DataFrame
final_group_totals.head()

# Export to Excel
final_group_totals.to_excel('group_nama_ward_member_size_paid.xlsx', index=False)